# Índice de Força Relativa (RSI - Relative Strenght Index)
O indicador RSI é bastante popular pois mede a força de um ativo pelas mudanças de preços de fechamento, sendo utilizado como um indicador oscilador.<br />
Os indicadores baseados em médias móveis são mais classificados como indicadores de tendências, e naturalmente emitem seus sinais com algum atraso.<br />
Indicadores do tipo oscilador emitem seus sinais antes das reversões.

In [ ]:
import pandas as pd
import numpy as np
import datetime
import yfinance as yf
import plotly
import plotly.offline as plo
import plotly.graph_objs as plg
import plotly.express as plx
import plotly.subplots as pls
from scipy.stats import norm

pd.options.mode.copy_on_write = True

### Funções de Configurações Gráficas
Funções para configurações base de gráficos e exibição.

In [ ]:
def Plot_Formato(fig, title):
    title={'text': title, 'xanchor': 'center', 'yanchor': 'bottom', 'y':0, 'x':0.5,}
    
    fig.update_layout(title=title, 
            xaxis_rangeslider_visible=False,  width=1280, height=720,
            xaxis_showgrid=True, xaxis_gridwidth=1, xaxis_gridcolor='#E8E8E8',
            yaxis_showgrid=True, yaxis_gridwidth=1, yaxis_gridcolor='#E8E8E8',
            plot_bgcolor='rgba(0,0,0,0)')
    
def Plot_Show(data, title):
    fig = plg.Figure(data=data)

    Plot_Formato(fig, title)

    fig.show()   

### Leitura de Dados de Ativos
Realiza a leitura online de dados de ativos financeiros.

In [ ]:
Acao = "CPLE3.SA"
Periodo_Inicio = datetime.datetime(2019,1,1)
Periodo_Termino = datetime.datetime(2021,2,10)

In [ ]:
Dados = yf.download(Acao, start=Periodo_Inicio, end=Periodo_Termino)

In [ ]:
Dados

### Análise dos Preços
Candlestick da série de preços do ativo.

In [ ]:
SeriePrecos = plg.Figure(plg.Scatter(x=Dados.index, y=Dados["Close", Acao]))
SeriePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(SeriePrecos, "Preços")

In [ ]:
CandlePrecos = plg.Figure(plg.Candlestick(x=Dados.index,
                                          open=Dados["Open", Acao],
                                          high=Dados["High", Acao],
                                          low=Dados["Low", Acao],
                                          close=Dados["Close", Acao]))
CandlePrecos.update_layout(yaxis_tickformat=".2f")
Plot_Show(CandlePrecos, "Preços")

### Cálculo do RSI e séries de sinais
O RSI é baseado em médias simples comumente para períodos de 14 dias, conforme a seguinte equiação:
- Ganho Médio = (Média Ganhos 14d * 13 + Média Atual) / 14 
- Perda Média = (Média Perdas 14d * 13 + Média Atual) / 14 
- Força Relativa = Ganho Médio / Perda Média 
- Índice = 100 - 100 / (1 + Força Relativa) 


O índice terá assim um valor entre 0 e 100, indicando:
- Regiões de Sobre Compra (ativo caro) na faixa de 70 a 100.
- Regiões de Sobre Venda (ativo barato) na faixa de 0 a 30.

In [ ]:
JanelaDeTempo = 14

In [ ]:
Dados["Retorno", Acao] = Dados["Close", Acao].pct_change().fillna(0)

- Separação dos Ganhos e Perdas

In [ ]:
Dados["Ganhos", Acao] = Dados["Retorno", Acao][Dados["Retorno", Acao] > 0]
Dados["Ganhos", Acao] = Dados["Ganhos", Acao].fillna(0)

In [ ]:
Dados["Perdas", Acao] = Dados["Retorno", Acao][Dados["Retorno", Acao] < 0] * (-1)
Dados["Perdas", Acao] = Dados["Perdas", Acao].fillna(0)

In [ ]:
Dados["GanhoMedio", Acao] = 0.00
Dados["PerdaMedia", Acao] = 0.00

- Média Simples dia-a-dia usando 14 dias anteriores

In [ ]:
Dados["GanhoMedio", Acao] = Dados["Ganhos", Acao].rolling(JanelaDeTempo).mean()

In [ ]:
Dados["PerdaMedia", Acao] = Dados["Perdas", Acao].rolling(JanelaDeTempo).mean()

- Cálculo do Índice

In [ ]:
for i in range(JanelaDeTempo, len(Dados)):
    Dados.iloc[i]["GanhoMedio", Acao] = (Dados.iloc[i-1]["GanhoMedio", Acao] * (JanelaDeTempo - 1) + Dados.iloc[i]["Ganhos", Acao]) / JanelaDeTempo
    Dados.iloc[i]["PerdaMedia", Acao] = (Dados.iloc[i-1]["PerdaMedia", Acao] * (JanelaDeTempo - 1) + Dados.iloc[i]["Perdas", Acao]) / JanelaDeTempo

Dados["RSI", Acao] = 100 - (100 / (1 + (Dados["GanhoMedio", Acao] / Dados["PerdaMedia", Acao])))

In [ ]:
Dados.head(20)

In [ ]:
SerieRSI = pls.make_subplots(rows=2, cols=1)
SerieRSI.add_trace(plg.Candlestick(x=Dados.index,
                                   open=Dados["Open", Acao],
                                   high=Dados["High", Acao],
                                   low=Dados["Low", Acao],
                                   close=Dados["Close", Acao],
                                   name="Preços",
                                   opacity=0.5), 1, 1)
SerieRSI.add_trace(plg.Scatter(mode="lines",
                               x=Dados.index,
                               y=Dados["RSI", Acao],
                               name="RSI"), 2, 1)
SerieRSI.update_layout(xaxis2 = dict(domain=[0, 1]),
                       yaxis2 = dict(domain=[0, 0.45],
                                     range=[0, 100]),
                      shapes=[{"xref": "x2",
                               "yref": "y2",
                               "x0": Dados.index[0],
                               "y0": 30,
                               "x1": Dados.index[-1],
                               "y1": 30,
                               "line": {"width": 1,
                                        "dash": "dot",
                                        "color": "green"}},
                             {"xref": "x2",
                               "yref": "y2",
                               "x0": Dados.index[0],
                               "y0": 70,
                               "x1": Dados.index[-1],
                               "y1": 70,
                               "line": {"width": 1,
                                        "dash": "dot",
                                        "color": "green"}}])
Plot_Show(SerieRSI, "Preços & RSI")